In [ ]:
# VIX-Adjusted Momentum (US)

Replicating [VIX-Adjusted Momentum](https://cssanalytics.wordpress.com/2014/07/29/vix-adjusted-momentum/)

In [ ]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")
source("goofy/misc.common.R")

indices <- Indices()

In [ ]:
startDate <- as.Date('1990-01-02')
endDate <- as.Date('2019-06-30')

vixDt <- indices$YahooFinanceTimeSeries() %>%
    filter(NAME == '^VIX' & TIME_STAMP >= startDate & TIME_STAMP <= endDate) %>%
    select(TIME_STAMP, CLOSE) %>%
    collect()
    
sp500Dt <- indices$YahooFinanceTimeSeries() %>%
    filter(NAME == '^GSPC' & TIME_STAMP >= startDate & TIME_STAMP <= endDate) %>%
    select(TIME_STAMP, CLOSE) %>%
    collect()

allXts <- merge(sp500Xts <- xts(sp500Dt$CLOSE, sp500Dt$TIME_STAMP), xts(vixDt$CLOSE, vixDt$TIME_STAMP))
names(allXts) <- c('SP500', 'VIX')


In [ ]:
allXts$SMA_200 <- SMA(allXts$SP500, 200)
allXts$D_RET <- dailyReturn(allXts$SP500)
allXts$D_RET_LAG_1 <- stats::lag(allXts$D_RET, -1) #signal is generated and traded at the close, so use next day's return

allXts <- na.omit(allXts)

print(head(allXts))
print(tail(allXts))